In [206]:
import importlib

from langchain_core.tools import Tool
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage

from microbe.knowledge import prepare_vectorstore as vectorstore
importlib.reload(vectorstore)

from microbe.cnf_api import nutrient_calculator
importlib.reload(nutrient_calculator)

from microbe.rag_model import simple_diet_model
importlib.reload(simple_diet_model)

from IPython.core.display import Markdown

In [207]:
models={["llama3.2:1b", "llama3.2:3b", "granite3-dense:2b", "granite3-dense:8b", "mistral", "gemma3:1b", "gemma3:4b"]}

model_id = models[0]
model_provider="ollama"
db_name = "../diet_vector_db"
knowledge_dir = "../knowledge"

In [208]:
db = vectorstore.create_vectorstore(
    db_name=db_name,
    knowledge_dir=knowledge_dir,
    force=False
)

retriever = db.as_retriever()


Total number of chunks: 12841
Document types found: {'microbiome', 'CFG', 'diet'}
Vectorstore already exists at ../diet_vector_db. Use force=True to delete it.
Vectorstore at ../diet_vector_db with 12841 documents


In [209]:
rag_model = simple_diet_model.SimpleDietModel(
    retriever=retriever,
    model_id=model_id,
    model_provider=model_provider,
)

In [210]:
def retrieve_docs(query):
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results])

def guideline_retriever_tool(input_str: str) -> str:
    """
    Input: user goal or phrase (e.g., "foods high in live microbes")
    Returns: text excerpts from vectorstore.
    """
    try:
        results = retrieve_docs(input_str.strip())  # Your retriever
        return f"📘 Relevant guideline excerpts:\n{results}"
    except Exception as e:
        return f"❌ Error retrieving guidelines: {str(e)}"


tools = [
    Tool(
        name="NutrientCalculator",
        func=nutrient_calculator.nutrient_calculator_tool,
        description="Use this when you need to compute calories, macronutrients, or nutrient content for a food item and its quantity (e.g., 'Butter, regular | 9.46g'). Always use this after proposing a modification."
    ),
    Tool(
        name="FoodSelector",
        func=nutrient_calculator.food_selector_tool,
        description="Use this to verify if a food exists in the CNF before recommending it. Input should be a food name like 'Almond butter'.. You must use it before suggesting any food substitution."
    ),
    Tool(
    name="GuidelineRetriever",
    func=guideline_retriever_tool,
    description="Use this to find foods high in live microbes when the user asks to increase such foods. Input can be a user goal or phrase like 'live microbe foods for kids'."
)
]

In [211]:
rag_model.set_tools(tools)

In [215]:
# Compose your dynamic prompts
system_prompt = """
You are a nutrition assistant that helps modify dietary recalls to better meet user goals.

**Always follow these steps:**
1. Use `GuidelineRetriever` to identify foods high in live microbes.
2. Use `FoodSelector` to confirm those candidate foods exist in the Canadian Nutrient File (CNF).
3. Use `NutrientCalculator` to evaluate the nutrient impact of replacing foods.
4. Make no more than 2–3 substitutions, and explain your reasoning.

**Always return:**
- A table showing before and after
- An explanation of the live microbe benefit and nutritional equivalence
- Only food items that exist in CNF
"""
user_prompt = """
A 6-year-old Canadian child provided the following dietary recall for consuming the following food items (food_identifier in NutrientCalculator) in the given quantities (amount in NutrientCalculator) in grams (measure_search_name in NutrientCalculator) for a day:
{
Butter, regular:9.46,
Bread, egg, (challah), toasted:74,
Peanut butter, smooth type, fat, sugar and salt added:16,
Sweets, honey, strained or extracted:21,
Water, mineral, \"POLAND SPRINGS\", bottled:284.16,
Grape, red or green (European type, such as Thompson seedless), adherent skin, raw:113.25,
Deli-meat, pepperoni:102,
Cracker, standard snack-type:31,
Deli-meat, pepperoni:89.25,
Fruit juice blend, 100% juice, with added Vitamin C:262.08,
Cookie, chocolate sandwich, cream filling, regular:8.4,
Endive, chicory, escarole, or romaine, raw:7.5,
Pork, cured, bacon, cooked, pan-fried:5,
Salad dressing, caesar dressing, regular:2.45,
Chicken, breast, with or without bone, roasted, skin not eaten:56,
Popcorn, air-popped, buttered:26,Orange juice, chilled, includes from concentrate:14.4
}.

Recommend what food items should be swapped with what so that the child's includes more live microbe foods, but the energy intake remains the same.
Present side by side the complete recalls (original and with modifications) in table format.
"""

# Initialize app with chat history containing both prompts
chat_history = [
    SystemMessage(content=system_prompt),
    # ✨ Few-shot example #1
    HumanMessage(content="A child consumed cheddar cheese | 15g and white bread | 36g. How can we improve this for gut health?"),
    AIMessage(content="""
                        **Thought**: Cheddar cheese may not contain live cultures. I want to find a food high in live microbes.

                        **Action**: GuidelineRetriever
                        **Input**: "foods high in live microbes"

                        **Observation**: Kefir and kimchi are fermented foods high in live microbes.

                        **Action**: FoodSelector
                        **Input**: "Kefir"

                        **Observation**: Kefir is available in CNF.

                        **Action**: NutrientCalculator
                        **Input**: "Kefir | 30g"

                        **Observation**: Kefir has similar calories to cheese and provides probiotics.

                        **Final Answer**:

                        | Original          | Modified |
                        |-------------------|----------|
                        | Cheddar cheese    | Kefir    |
                        | White bread       | Whole grain sourdough |

                        Substitutions increase live microbe intake while maintaining energy. Kefir and sourdough are microbiome-friendly and exist in CNF.
                    """),
    # ✨ Few-shot example #2
    HumanMessage(content="""
                        A 7-year-old consumed the following for breakfast:
                        'Pancakes with syrup | 50g ' and 'orange juice | 100g'
                        Please improve this breakfast to increase fiber intake without reducing calories.
                        """
                 ),
    AIMessage(content="""
                        **Thought**: Pancakes and syrup are low in fiber. I need to find a high-fiber food.

                        **Action**: GuidelineRetriever
                        **Input**: "high fiber foods for kids"

                        **Observation**: Oatmeal and berries are high in fiber.

                        **Action**: FoodSelector
                        **Input**: "Oatmeal"

                        **Observation**: Oatmeal is available in CNF.

                        **Action**: NutrientCalculator
                        **Input**: "Oatmeal | 50g"

                        **Observation**: Oatmeal has similar calories to pancakes and provides more fiber.

                        **Final Answer**:

                        | Original          | Modified |
                        |-------------------|----------|
                        | Pancakes with syrup| Oatmeal  |
                        | Orange juice      | Berries  |

                        Substitutions increase fiber intake while maintaining calories. Oatmeal and berries are nutritious and exist in CNF.
                    """),
    # ✨ Few-shot example #3
    #HumanMessage(content="""
    #                    A teenager had 'string cheese | 30g' and 'white crackers | 30g' for snack. Can you improve this to include more live microbe foods?
    #                    """
    #             ),
    #AIMessage(content=""")
    #                    **Thought**: String cheese may not have live cultures. I need to find a food high in live microbes.

    #                    **Action**: GuidelineRetriever
    #                    **Input**: "foods high in live microbes"

    #                    **Observation**: Yogurt and sauerkraut are high in live microbes.

    #                    **Action**: FoodSelector
    #                    **Input**: "Yogurt"

    #                    **Observation**: Yogurt is available in CNF.

    #                    **Action**: NutrientCalculator
    #                    **Input**: "Yogurt | 30g"

    #                    **Observation**: Yogurt has similar calories to cheese and provides probiotics.

    #                    **Final Answer**:

    #                    | Original          | Modified |
    #                    |-------------------|----------|
    #                    | String cheese     | Yogurt   |
    #                    | White crackers    | Whole grain crackers |

    #                    Substitutions increase live microbe intake while maintaining energy. Yogurt and whole grain crackers are microbiome-friendly and exist in CNF.
    #                """),
    # ✨ Few-shot example #4
    #HumanMessage(content=""")
    #                    A 9-year-old’s school lunch includes 'chocolate milk | 100g', 'granola bar | 75g', and "peanut butter sandwich | 90g". Goal: reduce added sugar.
    #                    """
    #             ),
    #AIMessage(content=""")
    #                    **Thought**: Chocolate milk and granola bars are high in added sugar. I need to find lower-sugar alternatives.

    #                    **Action**: GuidelineRetriever
    #                    **Input**: "low sugar foods for kids"

    #                    **Observation**: Unsweetened yogurt and whole grain bread are lower in sugar.

    #                    **Action**: FoodSelector
    #                    **Input**: "Unsweetened yogurt"

    #                    **Observation**: Unsweetened yogurt is available in CNF.

    #                    **Action**: NutrientCalculator
    #                    **Input**: "Unsweetened yogurt | 100g"

    #                    **Observation**: Unsweetened yogurt has similar calories to chocolate milk and less sugar.

    #                    **Final Answer**:

    #                    | Original          | Modified |
    #                    |-------------------|----------|
    #                    | Chocolate milk    | Unsweetened yogurt |
    #                    | Granola bar       | Whole grain bread |
    #                    | Peanut butter sandwich | Peanut butter on whole grain |

    #                    Substitutions reduce added sugar while maintaining energy. Unsweetened yogurt and whole grain options are healthier and exist in CNF.
    #                """),
    SystemMessage(content="---\nNow process a new user request below. Do NOT copy from previous examples. Think independently.\n---"),
    HumanMessage(content=user_prompt),
    AIMessage(content="**Thought**: Let me analyze this specific recall before suggesting substitutions.")
]

In [216]:
final_response = rag_model.invoke_model(messages=chat_history)

In [217]:
display(Markdown(final_response))

 

- Regular butter contains around 20g of fat, which might be a concern for health reasons. However, it's essential to note that there is no direct link between regular butter and live microbes.

- Sliced white bread (74g) is high in refined carbohydrates but does not contain significant amounts of live microorganisms.

The child has consumed:
- 9.46g of butter
- 74g of toasted egg bread
- 16g of smooth peanut butter
- 21g of honey and other sweet treats
- 284.16g of bottled water
- 113.25g of raw grapes
- 102g of deli-meat pepperoni (twice)
- 89.25g of deli-meat pepperoni (again)
- 262.08g of fruit juice blend with added Vitamin C
- 8.4g of chocolate sandwich cookies
- 56g of roasted chicken breast
- 26g of air-popped popcorn, buttered
- 14.4g of chilled orange juice

To increase the fiber intake without reducing calories and to promote live microbe consumption:

| **Original** | **Modified** |
|------------|-------------|
| Butter       | Oatmeal      |
| Egg bread   | Whole grain toast with avocado or nuts |
| Peanut butter | Apple slices  |
| Honey     | Dried apricots or prunes |
| Fruit juice | Carrot sticks or cucumber slices |
| Popcorn    | Air-popped popcorn, mixed nuts, or seeds |
| Orange juice| Cucumber slices |

Substitutions aim to increase fiber and live microbe intake while maintaining the original calorie count.

**Explanation**: The modifications include replacing butter with oatmeal, adding avocado or nuts to egg bread, increasing fruit consumption through dried apricots or prunes, using carrot sticks or cucumber slices instead of air-popped popcorn, and reducing caloric content by substituting orange juice with cucumber slices.

In [218]:
import os
from datetime import datetime

# Variables for filename
model = 'diet-rag'
date_str = datetime.now().isoformat(timespec='hours')
filename = f"../output/{model}-{date_str}"
# Notebook name (must match file exactly)
notebook_file = "diet-rag.ipynb"

# Export to PDF
exit_code = os.system(f'jupyter nbconvert --to pdf "{notebook_file}" --output "{filename}"')


# Check if export succeeded
if exit_code != 0:
    print("❌ Export failed. Check that LaTeX is installed and notebook filename is correct.")
else:
    print(f"✅ Exported successfully to {filename}")

✅ Exported successfully to ../output/diet-rag-2025-05-31T16


In [32]:
print(":".join(["ollama", "llama3.2:1b"]))

ollama:llama3.2:1b
